In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': '10.191.1.241', 'port': 9200}])

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
#@timing
def search(text):
    return es.search(index="tweets-enriquecer*", body=
        {
            "from" : 0, "size" : 500,
              "query": {
                "simple_query_string" : {
                    "query": text                }
              }
        })['hits']['hits']

In [7]:
textos = []
for res in search('esquina'):
    textos.append(res['_source']['tweet_content'])

In [9]:
import csv

with open('test_set_esquinas.csv', mode='w') as textos_file:
    t_writer = csv.writer(textos_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for texto in textos:
        t_writer.writerow([texto.encode('ascii', 'ignore').decode('ascii')])

In [20]:
import pandas as pd
test_set = pd.read_csv("test_set_esquinas.csv")

In [21]:
test_set = test_set.drop_duplicates()

In [22]:
test_set.to_csv('test_set_esquinas_clean.csv')

In [15]:
from datetime import datetime
from elasticsearch import Elasticsearch
es_local = Elasticsearch([{'host': 'elasticsearch', 'port': 9200}])
from elasticsearch.client import IndicesClient
ic = IndicesClient(es_local)
def analyzedText(text):
    results = ic.analyze(index="montevideo", body=
    {
      "text": text, 
        "analyzer":"calle_analyzer"
    })
    return ' '.join([token['token'] for token in results['tokens']])

In [18]:
with open('test_set_esquinas.csv', mode='w') as textos_file:
    t_writer = csv.writer(textos_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for texto in textos:
        t_writer.writerow([analyzedText(texto).encode('ascii', 'ignore').decode('ascii')])